### Calculating the number of sites used in the FYP - (2) Diversity calculations

In [2]:
# INSTALLING AND IMPORTING PACKAGES

#!pip install -q malariagen_data
#!pip install ipyleaflet

import numpy as np
import matplotlib.pyplot as plt
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})
import allel; print('scikit-allel', allel.__version__)
import malariagen_data
import pandas as pd
import pickle 

scikit-allel 1.3.5


In [3]:
# IMPORT API
# AG3 DATA ACCESS FROM GOOGLE CLOUD

ag3 = malariagen_data.Ag3(pre='True') # Pre=True is needed to include data from all data beyond 3.0 release/phase (3.0-3.8)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8
Results cache           : None
Cohorts analysis        : 20230223
Species analysis        : aim_20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 7.3.1
Client location         : Wales, GB
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

#### Calculating the number of sites used for the original chromosome positions

Given that the diversity_stats() function requires the region to be specified, you can use snp_calls() to count the number of SNPs in that region to get the number of sites that were used. This can be used to count the number of polymorphic sites too.

In [ ]:
# COUNTING THE NUMBER OF SITES USED TO COMPUTE THE DIVERSITY STATISTICS

# Conducting snp_calls for the desired samples
snps_X_4_2013 = ag3.snp_calls(cohort_size=70, region= "X", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")
snps_3R_4_2013 = ag3.snp_calls(cohort_size=70,region= "3R", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")
snps_X_int_2013 = ag3.snp_calls(cohort_size=70,region= "X", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")
snps_3R_int_2013 = ag3.snp_calls(cohort_size=70, region= "3R", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")

snps_X_4_2015 = ag3.snp_calls(cohort_size=70, region= "X", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")
snps_3R_4_2015 = ag3.snp_calls(cohort_size=70,region= "3R", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")
snps_X_int_2015 = ag3.snp_calls(cohort_size=70,region= "X", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")
snps_3R_int_2015 = ag3.snp_calls(cohort_size=70, region= "3R", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")

In [ ]:
# Computing genotype calls with the SNP datasets
snps_X_4_2013_gt = allel.GenotypeDaskArray(snps_X_4_2013["call_genotype"].data)
snps_3R_4_2013_gt = allel.GenotypeDaskArray(snps_3R_4_2013["call_genotype"].data)
snps_X_int_2013_gt = allel.GenotypeDaskArray(snps_X_int_2013["call_genotype"].data)
snps_3R_int_2013_gt = allel.GenotypeDaskArray(snps_3R_int_2013["call_genotype"].data)

snps_X_4_2015_gt = allel.GenotypeDaskArray(snps_X_4_2015["call_genotype"].data)
snps_3R_4_2015_gt = allel.GenotypeDaskArray(snps_3R_4_2015["call_genotype"].data)
snps_X_int_2015_gt = allel.GenotypeDaskArray(snps_X_int_2015["call_genotype"].data)
snps_3R_int_2015_gt = allel.GenotypeDaskArray(snps_3R_int_2015["call_genotype"].data)

In [1]:
# REMOVE MISSING GENOTYPES

def remove_missing(snps, gt):
    # Create a 2D boolean array where TRUE indicates missing genotypes 
    missing = gt.is_missing() 
    # Create a 1D boolean array where it is TRUE if any of the genotypes per row is TRUE i.e., missing genotype for even 1 indivdual
    missing = np.any(missing, axis=1)
    mask = missing.compute()
    # Create an index array of the variant positions with no missing genotypes
    index = np.where(mask==False)[0] 
    # Subset the snp dataset to only contain non-missing sites
    new_snps = snps.isel(variants=index)
    return new_snps


In [ ]:
# Filter the snps for non-missing snps only
snps_X_4_2013_no_nan = remove_missing(snps_X_4_2013, snps_X_4_2013_gt)
snps_X_int_2013_no_nan = remove_missing(snps_X_int_2013, snps_X_int_2013_gt)
snps_3R_4_2013_no_nan = remove_missing(snps_3R_4_2013, snps_3R_4_2013_gt)
snps_3R_int_2013_no_nan = remove_missing(snps_3R_int_2013, snps_3R_int_2013_gt)

snps_X_4_2015_no_nan = remove_missing(snps_X_4_2015, snps_X_4_2015_gt)
snps_X_int_2015_no_nan = remove_missing(snps_X_int_2015, snps_X_int_2015_gt)
snps_3R_4_2015_no_nan = remove_missing(snps_3R_4_2015, snps_3R_4_2015_gt)
snps_3R_int_2015_no_nan = remove_missing(snps_3R_int_2015, snps_3R_int_2015_gt)

In [ ]:
# EXPORT THE DATASETS USING AS .PKL

with open('snps_X_4_2013_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_X_4_2013_no_nan, f)

with open('snps_X_int_2013_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_X_int_2013_no_nan, f)
    
with open('snps_3R_4_2013_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_3R_4_2013_no_nan, f)

with open('snps_3R_int_2013_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_3R_int_2013_no_nan, f)
    
    
with open('snps_X_4_2015_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_X_4_2015_no_nan, f)

with open('snps_X_int_2015_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_X_int_2015_no_nan, f)
    
with open('snps_3R_4_2015_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_3R_4_2015_no_nan, f)

with open('snps_3R_int_2015_no_nan.pkl', 'wb') as f:
    pickle.dump(snps_3R_int_2015_no_nan, f)

In [ ]:
# RE-COMPUTE GENOTYPE CALLS

snps_X_4_2013_no_nan_gt = allel.GenotypeDaskArray(snps_X_4_2013_no_nan["call_genotype"].data)
snps_X_int_2013_no_nan_gt = allel.GenotypeDaskArray(snps_X_int_2013_no_nan["call_genotype"].data)
snps_3R_4_2013_no_nan_gt = allel.GenotypeDaskArray(snps_3R_4_2013_no_nan["call_genotype"].data)
snps_3R_int_2013_no_nan_gt = allel.GenotypeDaskArray(snps_3R_int_2013_no_nan["call_genotype"].data)

snps_X_4_2015_no_nan_gt = allel.GenotypeDaskArray(snps_X_4_2015_no_nan["call_genotype"].data)
snps_X_int_2015_no_nan_gt = allel.GenotypeDaskArray(snps_X_int_2015_no_nan["call_genotype"].data)
snps_3R_4_2015_no_nan_gt = allel.GenotypeDaskArray(snps_3R_4_2015_no_nan["call_genotype"].data)
snps_3R_int_2015_no_nan_gt = allel.GenotypeDaskArray(snps_3R_int_2015_no_nan["call_genotype"].data)

In [ ]:
# COMPUTE ALLELE COUNTS

snps_X_4_2013_no_nan_ac = snps_X_4_2013_no_nan_gt.count_alleles(max_allele=3).compute()
snps_X_int_2013_no_nan_ac = snps_X_int_2013_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_4_2013_no_nan_ac = snps_3R_4_2013_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_int_2013_no_nan_ac = snps_3R_int_2013_no_nan_gt.count_alleles(max_allele=3).compute()

snps_X_4_2015_no_nan_ac = snps_X_4_2015_no_nan_gt.count_alleles(max_allele=3).compute()
snps_X_int_2015_no_nan_ac = snps_X_int_2015_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_4_2015_no_nan_ac = snps_3R_4_2015_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_int_2015_no_nan_ac = snps_3R_int_2015_no_nan_gt.count_alleles(max_allele=3).compute()

In [ ]:
# COUNT THE NUMBER OF VARIANTS (BIALLELIC, MULTIALLELIC AND FIXED)

print("Number of variants 2013 (X_4):", len(snps_X_4_2013_no_nan.variants)) # Total number of sites
print("Number of fixed sites 2013 (X_4):", np.count_nonzero(~snps_X_4_2013_no_nan_ac.is_variant())) #~ needed to count where it is false
print("Number of segregating SNPs 2013 (X_4):", (snps_X_4_2013_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (X_4):", snps_X_4_2013_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (X_4):", (snps_X_4_2013_no_nan_ac.is_segregating().sum()- snps_X_4_2013_no_nan_ac.is_biallelic().sum()))
# Biallelic + multiallelic = segregating

print("\nNumber of variants 2015 (X_4):", len(snps_X_4_2015_no_nan.variants))
print("Number of fixed sites 2015 (X_4):", np.count_nonzero(~snps_X_4_2015_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (X_4):", (snps_X_4_2015_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (X_4):", snps_X_4_2015_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (X_4):", (snps_X_4_2015_no_nan_ac.is_segregating().sum()- snps_X_4_2015_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2013 (X_int):", len(snps_X_int_2013_no_nan.variants))
print("Number of fixed sites 2013 (X_int):", np.count_nonzero(~snps_X_int_2013_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2013 (X_int):", (snps_X_int_2013_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (X_int):", snps_X_int_2013_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (X_int):", (snps_X_int_2013_no_nan_ac.is_segregating().sum()- snps_X_int_2013_no_nan_ac.is_biallelic().sum()))


print("\nNumber of variants 2015 (X_int):", len(snps_X_int_2015_no_nan.variants))
print("Number of fixed sites 2015 (X_int):", np.count_nonzero(~snps_X_int_2015_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (X_int):", (snps_X_int_2015_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (X_int):", snps_X_int_2015_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (X_int):", (snps_X_int_2015_no_nan_ac.is_segregating().sum()- snps_X_int_2015_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2013 (3R_4):", len(snps_3R_4_2013_no_nan.variants))
print("Number of fixed sites 2013 (3R_4):", np.count_nonzero(~snps_3R_4_2013_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2013 (3R_4):", (snps_3R_4_2013_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (3R_4):", snps_3R_4_2013_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (3R_4):", (snps_3R_4_2013_no_nan_ac.is_segregating().sum()- snps_3R_4_2013_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2015 (3R_4):", len(snps_3R_4_2015_no_nan.variants))
print("Number of fixed sites 2015 (3R_4):", np.count_nonzero(~snps_3R_4_2015_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (3R_4):", (snps_3R_4_2015_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (3R_4):", snps_3R_4_2015_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (3R_4):", (snps_3R_4_2015_no_nan_ac.is_segregating().sum()- snps_3R_4_2015_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2013 (3R_int):", len(snps_3R_int_2013_no_nan.variants))
print("Number of fixed sites 2013 (3R_int):", np.count_nonzero(~snps_3R_int_2013_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2013 (3R_int):", (snps_3R_int_2013_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (3R_int):", snps_3R_int_2013_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (3R_int):", (snps_3R_int_2013_no_nan_ac.is_segregating().sum()- snps_3R_int_2013_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2015 (3R_int):", len(snps_3R_int_2015_no_nan.variants))
print("Number of fixed sites 2015 (3R_int):", np.count_nonzero(~snps_3R_int_2015_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (3R_int):", (snps_3R_int_2015_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (3R_int):", snps_3R_int_2015_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (3R_int):", (snps_3R_int_2015_no_nan_ac.is_segregating().sum()- snps_3R_int_2015_no_nan_ac.is_biallelic().sum()))

#### For the new chromosome positions

In [ ]:
# Conducting snp_calls for the desired samples
snps_X_4_2013_new = ag3.snp_calls(cohort_size=70, region= "X:1-14000000", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")
snps_3R_4_2013_new = ag3.snp_calls(cohort_size=70,region= "3R:1-35000000", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")
snps_X_int_2013_new = ag3.snp_calls(cohort_size=70,region= "X:1-14000000", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")
snps_3R_int_2013_new = ag3.snp_calls(cohort_size=70, region= "3R:1-35000000", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2013) and (location == 'Sokourani (Niono)')")

In [ ]:
snps_X_4_2015_new = ag3.snp_calls(cohort_size=70, region= "X:1-14000000", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")
snps_3R_4_2015_new = ag3.snp_calls(cohort_size=70,region= "3R:1-35000000", site_mask="gamb_colu", site_class="CDS_DEG_4", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")
snps_X_int_2015_new = ag3.snp_calls(cohort_size=70,region= "X:1-14000000", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")
snps_3R_int_2015_new = ag3.snp_calls(cohort_size=70, region= "3R:1-35000000", site_mask="gamb_colu", site_class="INTERGENIC", sample_query = "(country == 'Mali') and (taxon == 'coluzzii') and (year == 2015) and (location == 'Sokourani (Niono)')")

In [ ]:
# Computing genotype calls with the SNP datasets
snps_X_4_2013_new_gt = allel.GenotypeDaskArray(snps_X_4_2013_new["call_genotype"].data)
snps_3R_4_2013_new_gt = allel.GenotypeDaskArray(snps_3R_4_2013_new["call_genotype"].data)
snps_X_int_2013_new_gt= allel.GenotypeDaskArray(snps_X_int_2013_new["call_genotype"].data)
snps_3R_int_2013_new_gt = allel.GenotypeDaskArray(snps_3R_int_2013_new["call_genotype"].data)

snps_X_4_2015_new_gt = allel.GenotypeDaskArray(snps_X_4_2015_new["call_genotype"].data)
snps_3R_4_2015_new_gt  = allel.GenotypeDaskArray(snps_3R_4_2015_new["call_genotype"].data)
snps_X_int_2015_new_gt  = allel.GenotypeDaskArray(snps_X_int_2015_new["call_genotype"].data)
snps_3R_int_2015_new_gt  = allel.GenotypeDaskArray(snps_3R_int_2015_new["call_genotype"].data)

In [ ]:
# Filter the snps for non-missing snps only
snps_X_4_2013_new_no_nan = remove_missing(snps_X_4_2013_new, snps_X_4_2013_new_gt)
snps_X_int_2013_new_no_nan = remove_missing(snps_X_int_2013_new, snps_X_int_2013_new_gt)
snps_3R_4_2013_new_no_nan = remove_missing(snps_3R_4_2013_new, snps_3R_4_2013_new_gt)
snps_3R_int_2013_new_no_nan = remove_missing(snps_3R_int_2013_new, snps_3R_int_2013_new_gt)

snps_X_4_2015_new_no_nan = remove_missing(snps_X_4_2015_new, snps_X_4_2015_new_gt)
snps_X_int_2015_new_no_nan = remove_missing(snps_X_int_2015_new, snps_X_int_2015_new_gt)
snps_3R_4_2015_new_no_nan = remove_missing(snps_3R_4_2015_new, snps_3R_4_2015_new_gt)
snps_3R_int_2015_new_no_nan = remove_missing(snps_3R_int_2015_new, snps_3R_int_2015_new_gt)

In [ ]:
# RE-COMPUTE GENOTYPE CALLS

snps_X_4_2013_new_no_nan_gt = allel.GenotypeDaskArray(snps_X_4_2013_new_no_nan["call_genotype"].data)
snps_X_int_2013_new_no_nan_gt = allel.GenotypeDaskArray(snps_X_int_2013_new_no_nan["call_genotype"].data)
snps_3R_4_2013_new_no_nan_gt = allel.GenotypeDaskArray(snps_3R_4_2013_new_no_nan["call_genotype"].data)
snps_3R_int_2013_new_no_nan_gt = allel.GenotypeDaskArray(snps_3R_int_2013_new_no_nan["call_genotype"].data)

snps_X_4_2015_new_no_nan_gt = allel.GenotypeDaskArray(snps_X_4_2015_new_no_nan["call_genotype"].data)
snps_X_int_2015_new_no_nan_gt = allel.GenotypeDaskArray(snps_X_int_2015_new_no_nan["call_genotype"].data)
snps_3R_4_2015_new_no_nan_gt = allel.GenotypeDaskArray(snps_3R_4_2015_new_no_nan["call_genotype"].data)
snps_3R_int_2015_new_no_nan_gt = allel.GenotypeDaskArray(snps_3R_int_2015_new_no_nan["call_genotype"].data)

In [ ]:
# COMPUTE ALLELE COUNTS

snps_X_4_2013_new_no_nan_ac = snps_X_4_2013_new_no_nan_gt.count_alleles(max_allele=3).compute()
snps_X_int_2013_new_no_nan_ac = snps_X_int_2013_new_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_4_2013_new_no_nan_ac = snps_3R_4_2013_new_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_int_2013_new_no_nan_ac = snps_3R_int_2013_new_no_nan_gt.count_alleles(max_allele=3).compute()

snps_X_4_2015_new_no_nan_ac = snps_X_4_2015_new_no_nan_gt.count_alleles(max_allele=3).compute()
snps_X_int_2015_new_no_nan_ac = snps_X_int_2015_new_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_4_2015_new_no_nan_ac = snps_3R_4_2015_new_no_nan_gt.count_alleles(max_allele=3).compute()
snps_3R_int_2015_new_no_nan_ac = snps_3R_int_2015_new_no_nan_gt.count_alleles(max_allele=3).compute()

In [ ]:
# COUNT THE NUMBER OF VARIANTS (BIALLELIC, MULTIALLELIC AND FIXED) FOR THE NEW CHROMOSOME POSITIONS

print("Number of variants 2013 (X_4):", len(snps_X_4_2013_new_no_nan)) # Total number of sites
print("Number of fixed sites 2013 (X_4):", np.count_nonzero(~snps_X_4_2013_new_no_nan_ac.is_variant())) #~ needed to count where it is false
print("Number of segregating SNPs 2013 (X_4):", (snps_X_4_2013_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (X_4):", snps_X_4_2013_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (X_4):", (snps_X_4_2013_new_no_nan_ac.is_segregating().sum()- snps_X_4_2013_new_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2015 (X_4):", len(snps_X_4_2015_new_no_nan))
print("Number of fixed sites 2015 (X_4):", np.count_nonzero(~snps_X_4_2015_new_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (X_4):", (snps_X_4_2015_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (X_4):", snps_X_4_2015_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (X_4):", (snps_X_4_2015_new_no_nan_ac.is_segregating().sum()- snps_X_4_2015_new_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2013 (X_int):", len(snps_X_int_2013_new_no_nan))
print("Number of fixed sites 2013 (X_int):", np.count_nonzero(~snps_X_int_2013_new_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2013 (X_int):", (snps_X_int_2013_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (X_int):", snps_X_int_2013_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (X_int):", (snps_X_int_2013_new_no_nan_ac.is_segregating().sum()- snps_X_int_2013_new_no_nan_ac.is_biallelic().sum()))


print("\nNumber of variants 2015 (X_int):", len(snps_X_int_2015_new_no_nan))
print("Number of fixed sites 2015 (X_int):", np.count_nonzero(~snps_X_int_2015_new_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (X_int):", (snps_X_int_2015_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (X_int):", snps_X_int_2015_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (X_int):", (snps_X_int_2015_new_no_nan_ac.is_segregating().sum()- snps_X_int_2015_new_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2013 (3R_4):", len(snps_3R_4_2013_new_no_nan))
print("Number of fixed sites 2013 (3R_4):", np.count_nonzero(~snps_3R_4_2013_new_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2013 (3R_4):", (snps_3R_4_2013_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (3R_4):", snps_3R_4_2013_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (3R_4):", (snps_3R_4_2013_new_no_nan_ac.is_segregating().sum()- snps_3R_4_2013_new_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2015 (3R_4):", len(snps_3R_4_2015_new_no_nan))
print("Number of fixed sites 2015 (3R_4):", np.count_nonzero(~snps_3R_4_2015_new_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (3R_4):", (snps_3R_4_2015_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (3R_4):", snps_3R_4_2015_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (3R_4):", (snps_3R_4_2015_new_no_nan_ac.is_segregating().sum()- snps_3R_4_2015_new_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2013 (3R_int):", len(snps_3R_int_2013_new_no_nan))
print("Number of fixed sites 2013 (3R_int):", np.count_nonzero(~snps_3R_int_2013_new_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2013 (3R_int):", (snps_3R_int_2013_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2013 (3R_int):", snps_3R_int_2013_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2013 (3R_int):", (snps_3R_int_2013_new_no_nan_ac.is_segregating().sum()- snps_3R_int_2013_new_no_nan_ac.is_biallelic().sum()))

print("\nNumber of variants 2015 (3R_int):", len(snps_3R_int_2015_new_no_nan))
print("Number of fixed sites 2015 (3R_int):", np.count_nonzero(~snps_3R_int_2015_new_no_nan_ac.is_variant()))
print("Number of segregating SNPs 2015 (3R_int):", (snps_3R_int_2015_new_no_nan_ac.is_segregating().sum()))
print("Number of biallelic SNPs 2015 (3R_int):", snps_3R_int_2015_new_no_nan_ac.is_biallelic().sum())
print("Number of multiallelic SNPs 2015 (3R_int):", (snps_3R_int_2015_new_no_nan_ac.is_segregating().sum()- snps_3R_int_2015_new_no_nan_ac.is_biallelic().sum()))